In [5]:
#libraries
import requests
from io import BytesIO

import time
import os
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import json
import re
import ast
import base64
import pickle

from minio import Minio
from minio.error import S3Error

In [6]:
def get_latest_file_with_extension(extension):
    objects = client.list_objects(bucket_name, prefix=prefix, recursive=True)
    files = [obj for obj in objects if obj.object_name.endswith(extension)]
    if not files:
        return None
    files.sort(key=lambda x: x.last_modified, reverse=True)
    return files[0]
def show_json(obj_key,output):
    response = client.get_object(bucket_name, obj_key)
    json_bytes = response.read()
    # Save json file in output/
    os.makedirs(output, exist_ok=True)
    local_path = os.path.join(output, os.path.basename(obj_key))
    with open(local_path, "wb") as f:
        f.write(json_bytes)
    print(f"JSON file saved in: {local_path}")
    # Show content to json file
    print(f"Content of {obj_key}:\n")
    print(json_bytes.decode('utf-8'))

In [7]:

minio_credencials= os.getenv("MINIO_CREDENCIALS","minio_token1.pkl")

if not minio_credencials.endswith(".pkl"):
    minio_credencials += ".pkl"
# Load configuration from .pkl file
with open(minio_credencials, "rb") as f:
    config = pickle.load(f)
# Create MinIO Client
client = Minio(
    endpoint=config["endpoint"],
    access_key=config["access_key"],
    secret_key=config["secret_key"],
    secure= True
)

In [8]:
#Necessary environment variables

bucket_name= os.getenv("BUCKET_NAME","")
out= os.getenv("PREFIX","")
prefix=out+"/"
output= os.getenv("OUTPUT","output")
download_dir = "./temporal"
os.makedirs(download_dir, exist_ok=True)

In [12]:
print(f"Waiting for new .json file in {bucket_name}/{prefix}...")

last_seen_json = get_latest_file_with_extension('.json')
last_json_time = last_seen_json.last_modified if last_seen_json else None
while True:
    current_json = get_latest_file_with_extension('.json')
    new_json = (not last_seen_json or last_seen_json.last_modified > last_json_time)
    if current_json and new_json:
        print(f"Files detected: {current_json['Key']}")

        # Download and show json
        show_json(current_json['Key'],output)
        break
    time.sleep(5)

Waiting for new .json file in phytoplankton-classifier/output/...


KeyboardInterrupt: 